## UK Biobank Genomic QC

This notebook summarizes the processing steps we performed for variant and sample QC in the UK biobank datasets, as well as the steps needed to prepare the data for association testing implemented in BOLT-LMM.

### Load Hail

In [ ]:
import seaborn as sns
import hail as hl
import os
from gnomad.utils.liftover import *
from gnomad.utils.annotations import *
from gnomad.sample_qc.pipeline import *
from gnomad.sample_qc.ancestry import *

tmp = "/mnt/grid/janowitz/home/skleeman/tmp2"
os.environ["SPARK_LOCAL_DIRS"]=tmp

os.environ["PYSPARK_SUBMIT_ARGS"] ="--driver-memory 200g --executor-memory 2g pyspark-shell"

hl.init(default_reference='GRCh38', master='local[16]',min_block_size=128, local_tmpdir=tmp, tmp_dir=tmp)

### Merge PLINK genotype data

In [ ]:
%%bash

plink \
  --bed /mnt/grid/ukbiobank/data/genotypes/plink/ukb_cal_chr1_v2.bed \
  --bim /mnt/grid/janowitz/home/skleeman/ukbiobank/rawdata/ukb_snp_chr1_v2.bim \
  --fam /mnt/grid/ukbiobank/data/Application58510/rawdata/ukb58510_cal_chr1_v2_s488250.fam \
  --merge-list ~/list_beds.txt \
  --make-bed --out ukb_merged \

### Import PLINK data into Hail

In [ ]:
bed = '/mnt/grid/ukbiobank/data/Application58510/skleeman/merged_plink/ukb_merged.bed'
bim = '/mnt/grid/ukbiobank/data/Application58510/skleeman/merged_plink/ukb_merged.bim'
fam = '/mnt/grid/ukbiobank/data/Application58510/skleeman/merged_plink/ukb_merged.fam'

ukb = hl.import_plink(bed = bed, bim = bim, fam = fam, reference_genome='GRCh37',
                     min_partitions = 150)

ukb.write('/mnt/grid/ukbiobank/data/Application58510/skleeman/ukb_merged_raw.mt', overwrite=True) #Save raw data

### Dataset quality control

We filter to high-quality SNP, LD prune across genotyped data in UKB then merge with a reference dataset (1000 Genomes / Human Diversity Genome Project) using the callset provided by gnomad.

In [ ]:
#Import UKB raw data
ukb = hl.read_matrix_table('/mnt/grid/ukbiobank/data/Application58510/skleeman/ukb_merged_raw.mt')
print(ukb.count()) #How many variants to start with?
ukb = default_lift_data(ukb) #GNOMAD pipeline for liftover, including reverse complement on negative strand

#Filtering

ukb = hl.variant_qc(ukb) #Default Hail variant QC pipeline
ukb = ukb.filter_rows(hl.len(ukb.alleles) == 2) #Biallelic SNPs only
ukb = ukb.filter_rows(ukb.ref_allele_mismatch == False) #Remove alleles with reference mismatch ('allele flips')
ukb = ukb.filter_rows(ukb.variant_qc.AF[1] > 0.001) #MAF > 0.1%
ukb = ukb.filter_rows(ukb.variant_qc.call_rate > 0.99) #Filter by call rate > 99%
ukb = ukb.checkpoint('/mnt/grid/ukbiobank/data/Application58510/skleeman/ukb_grch38_filtered.mt', overwrite=True) #Save filtered MT

#Exclude LD intervals from plinkQC package
intervals = hl.import_bed('/mnt/grid/janowitz/home/skleeman/ukbiobank/cancergwas/remove_ld_grch38.bed',
                         reference_genome='GRCh38')
ukb = ukb.filter_rows(hl.is_defined(intervals[ukb.locus]),keep=False)

#LD pruning in PLINK (not working in Hail due to bug)
hl.export_plink(ukb, '/mnt/grid/ukbiobank/data/Application58510/skleeman/pre_ld_plink',
                fam_id = ukb.fam_id, ind_id = ukb.s, pat_id = ukb.pat_id, mat_id = ukb.mat_id,
                is_female = ukb.is_female)

commands = '''
plink --bfile /mnt/grid/ukbiobank/data/Application58510/skleeman/pre_ld_plink \
--indep-pairwise 50 5 0.2 --out /mnt/grid/ukbiobank/data/Application58510/skleeman/prune --threads 96 --memory 50000 \
--allow-extra-chr

plink --bfile /mnt/grid/ukbiobank/data/Application58510/skleeman/pre_ld_plink \
--extract /mnt/grid/ukbiobank/data/Application58510/skleeman/prune.prune.in \
--make-bed --out /mnt/grid/ukbiobank/data/Application58510/skleeman/post_ld_plink --threads 96 --memory 50000 \
--allow-extra-chr
'''

output = subprocess.check_output(commands, shell=True)
print(output)

#Import output of LD pruning back into Hail
bed = '/mnt/grid/ukbiobank/data/Application58510/skleeman/post_ld_plink.bed'
bim = '/mnt/grid/ukbiobank/data/Application58510/skleeman/post_ld_plink.bim'
fam = '/mnt/grid/ukbiobank/data/Application58510/skleeman/post_ld_plink.fam'

ukb = hl.import_plink(bed = bed, bim = bim, fam = fam, reference_genome='GRCh38',
                     min_partitions = 150)
print(ukb.count())
ukb = ukb.checkpoint('/mnt/grid/ukbiobank/data/Application58510/skleeman/ukb_grch38_pruned.mt', overwrite=True) #Save filtered MT

#Import reference data (1000G/HDGP)
this_ref = hl.read_matrix_table('/mnt/grid/janowitz/rdata_norepl/ref.mt') #Dense form (!!!)

this_ref = hl.variant_qc(this_ref) #Default Hail variant QC pipeline
this_ref = this_ref.filter_rows((hl.len(this_ref.alleles) == 2) & hl.is_snp(this_ref.alleles[0], this_ref.alleles[1]))
this_ref = this_ref.filter_rows(this_ref.variant_qc.call_rate > 0.95) #Filter by call rate > 95%
this_ref = this_ref.naive_coalesce(250)

this_ref = this_ref.checkpoint('/mnt/grid/janowitz/home/references/1k_hgdp/ref_gnomadfilters.mt', overwrite=True) #Save filtered MT

#Intersect to variants present in both datasets
ukbb_in_ref = ukb.filter_rows(hl.is_defined(this_ref.rows()[ukb.row_key]))
print('sites in ref and UKBB data, inds in UKBB: ' + str(ukbb_in_ref.count()))

ref_in_ukbb = this_ref.filter_rows(hl.is_defined(ukb.rows()[this_ref.row_key]))
print('sites in ref and UKBB data, inds in ref: ' + str(ref_in_ukbb.count()))

#Save intersected data
ref_in_ukbb.write('/mnt/grid/janowitz/home/references/1k_hgdp/ref_intersect.mt', overwrite=True)
ukbb_in_ref.write('/mnt/grid/ukbiobank/data/Application58510/skleeman/ukb_grch38_intersect.mt',overwrite=True)

#### Ancestry analysis

Implementation based on gnomad QC methododogy in their github repo.

##### Step 1

In [ ]:
this_ref = hl.read_matrix_table('/mnt/grid/janowitz/home/references/1k_hgdp/ref_intersect.mt')
ukb = hl.read_matrix_table('/mnt/grid/ukbiobank/data/Application58510/skleeman/ukb_grch38_intersect.mt')


#Find related individuals in 1000K/HGDP set

relatedness_ht = hl.pc_relate(this_ref.GT, 0.01, k=10, min_kinship=0.05, block_size=512)

related_samples_to_remove_ref = hl.maximal_independent_set(relatedness_ht.i, relatedness_ht.j, False)


#Run PCA

#--> Reference, label with inferred populations, exclude relateds
_, scores_pca_ref, loadings_pca_ref = run_pca_with_relateds(this_ref, related_samples_to_remove_ref, 
                                                               n_pcs=10, autosomes_only=True)

#--> UKB

scores_pca_ukb = pc_project(mt = ukb, loadings_ht = loadings_pca_ref)

scores_pca_ref.write("/mnt/grid/janowitz/home/references/1k_hgdp/scores_pca_ref.ht", overwrite=True)
scores_pca_ukb.write("/mnt/grid/ukbiobank/data/Application58510/skleeman/scores_pca_ukb.ht", overwrite=True)
related_samples_to_remove_ref.write("/mnt/grid/janowitz/home/references/1k_hgdp/related_remove_ref.ht",overwrite=True)

##### Step 2

In [ ]:
scores_pca_ref = hl.read_table("/mnt/grid/janowitz/home/references/1k_hgdp/scores_pca_ref.ht")
scores_pca_ukb = hl.read_table("/mnt/grid/ukbiobank/data/Application58510/skleeman/scores_pca_ukb.ht")
this_ref = hl.read_matrix_table('/mnt/grid/janowitz/home/references/1k_hgdp/ref_intersect.mt')

merge = scores_pca_ref.union(scores_pca_ukb)

merge = merge.annotate(
    training_pop=this_ref.cols()[merge.key].labeled_subpop)

recode = pd.read_excel('/mnt/grid/janowitz/home/references/1k_hgdp/recode.xlsx')
recode_ht = hl.Table.from_pandas(recode, key='labeled_subpop')

merge = merge.annotate(
    training_pop=recode_ht[merge.training_pop].superpop)

predictions_ref, classifer_rf_ref = assign_population_pcs(merge, pc_cols = merge.scores, known_col = 'training_pop', seed=501, min_prob = 0.70, missing_label='Other')

ukb_predictions = predictions_ref.semi_join(scores_pca_ukb) #Subset UKB samples

ukb_predictions.write("/mnt/grid/ukbiobank/data/Application58510/skleeman/ukb_predictions.ht", overwrite=True)

##### Analyze results (table)

In [ ]:
ukb_predictions_pd = ukb_predictions.to_pandas()
ukb_predictions_pd = ukb_predictions_pd[["s", "pop"]]
ukb_predictions_pd = ukb_predictions_pd[ukb_predictions_pd['pop'] != "Other"]
ukb_predictions_pd['pop'].value_counts()

##### Analyze results (UMAP projection)

In [ ]:
import umap
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

ukb_predictions = hl.read_table("/mnt/grid/ukbiobank/data/Application58510/skleeman/ukb_predictions.ht")

ukb_predictions_pd = ukb_predictions.to_pandas()

data = pd.DataFrame(ukb_predictions_pd['pca_scores'].to_list())

proj_umap_pca = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.5, random_state=42).fit_transform(data)

umap = pd.DataFrame(proj_umap_pca)
umap['pop'] = ukb_predictions_pd['pop']

plt.figure(figsize=(10,10))
sns.color_palette("bright")
sns.scatterplot(data=umap, x=0, y=1, hue="pop", s=1, alpha = 0.7, marker ='o')
plt.show()

### Prepare for GWAS

#### Filter genotype data for GWAS input

In [ ]:
#Import UKB data in PLINK form

ukb = hl.read_matrix_table('/mnt/grid/ukbiobank/data/Application58510/skleeman/ukb_merged_raw.mt')

print(ukb.count()) #How many variants to start with?

#Filtering - based partly on https://github.com/Nealelab/UK_Biobank_GWAS/blob/master/imputed-v2-gwas/README.md

ukb = hl.variant_qc(ukb) #Default Hail variant QC pipeline
ukb = ukb.filter_rows(ukb.variant_qc.AF[1] > 0.01) #MAF > 1%
ukb = ukb.filter_rows(ukb.variant_qc.call_rate > 0.95) #Filter by call rate > 95%

print(ukb.count()) #How many variants are left?

ukb.write('/mnt/grid/ukbiobank/data/Application58510/skleeman/ukb_grch37_filtered_forgwas.mt', overwrite=True) #Save pruned MT

#### Generate covariate dataset from UKB phenotypes

See GWAS_prep.R. This R script used to define phenotype (cortiscore), provide covariates (age, sex), remove samples not included in imputation, remove samples with sex chromosomal aneuploidy, remove outliers for missingness/heterozygosity rate, remove discordant sex.

#### Prepare for BOLT-LMM

Prepare covariates and phenotype input in each super-population (defined as above). Principal components run on all SNPs (fastPCA implementation used for EUR subset due to very large sample size), this is as per BOLT-LMM paper.

In [ ]:
#Pre-process ancestry predictions, merge with phenotype data
ukb_predictions = hl.read_table("/mnt/grid/ukbiobank/data/Application58510/skleeman/ukb_predictions.ht")

ukb_predictions_pd = ukb_predictions.to_pandas()
ukb_predictions_pd = ukb_predictions_pd[["s", "pop"]]
ukb_predictions_pd = ukb_predictions_pd[ukb_predictions_pd['pop'] != "Other"]
print(ukb_predictions_pd['pop'].value_counts(), flush=True)

phenotypes = pd.read_csv('/mnt/grid/janowitz/home/skleeman/ukbiobank/cancergwas/phenotype_input_cystatinc_gwas.csv')
phenotypes = phenotypes.drop(phenotypes.columns[0], axis=1)
phenotypes = phenotypes.rename(columns={'id': 's'})
phenotypes['s'] = phenotypes.s.astype(int)
ukb_predictions_pd['s'] = ukb_predictions_pd.s.astype(int)

phenotypes = phenotypes.merge(ukb_predictions_pd, how='inner', on='s')

populations = phenotypes['pop'].unique().tolist()

#Import GRCh37 data (filtered to call rate, MAF, no H-W filtering at present)
ukb_gwas = hl.read_matrix_table('/mnt/grid/ukbiobank/data/Application58510/skleeman/ukb_grch37_filtered_forgwas.mt')

#For each population, run PCA using FastPCA implemented in PLINK2, save covariate file, saved filtered binaries

for pop in populations:
    print(pop, flush=True)
    phenotypes['s'] = phenotypes.s.astype(str)
    
    #Subset covariate file, make suitable for BOLT-LMM
    
    subset = phenotypes[phenotypes['pop'] == pop]
    
    frame = { 'FID': subset['s'], 'IID': subset['s'], 'cortiscore': subset['cortiscore'], 'age': subset['age_when_attended_assessment_centre_f21003'],
        'age_squared': subset['age_when_attended_assessment_centre_f21003']**2, 'sex': subset['sex_f31_0_0'], 'steroid_factor': subset['steroid_factor'],
        'height': subset['standing_height_f50'], 'weight': subset['weight_f21002'], 'array': subset['genotype_measurement_batch_f22000_0_0'],
        'centre': subset['uk_biobank_assessment_centre_f54_0_0'], 'egfr_cystatin': subset['egfr_cystatin'],
        'egfr_creatinine': subset['egfr_creatinine']} 

    covariates = pd.DataFrame(frame) 
    
    #Subset GRCH37 version of UK biobank dataset
    
    subset_ht = hl.Table.from_pandas(subset['s'].to_frame(), key='s') 
    ukb_pop_gwas = ukb_gwas.filter_cols(hl.is_defined(subset_ht[ukb_gwas.col_key]))


    #Save filtered PLINK file (GRCH37, include rsid)
    
    folder = '/mnt/grid/ukbiobank/data/Application58510/skleeman/gwas_cystatinc/' + pop + '/filtered_grch37'
    
    if not os.path.exists(folder + '.bim'):
        hl.export_plink(ukb_pop_gwas, folder,
                    fam_id = ukb_pop_gwas.fam_id, ind_id = ukb_pop_gwas.s, pat_id = ukb_pop_gwas.pat_id, mat_id = ukb_pop_gwas.mat_id,
                    is_female = ukb_pop_gwas.is_female, varid = ukb_pop_gwas.rsid, pheno = -9)
    
    #Run PCA on all filtered variants, per population
    
    output = '/mnt/grid/ukbiobank/data/Application58510/skleeman/gwas_cystatinc/' + pop + '/pca'
    eigenvec = output + '.eigenvec'
    
    if not os.path.exists(eigenvec):
        print('Start PCA', flush=True)
        if pop == "EUR":  #PLINK PCA, approx
            commands = "plink2 --bfile %s --pca 20 approx --out %s --threads 25" % (folder, output)
        else: #PLINK PCA, not approx
            commands = "plink2 --bfile %s --pca 20 --out %s --threads 25" % (folder, output)
        output = subprocess.check_output(commands, shell=True)
        print(output)
        print('Finish PCA', flush=True)
    
    pca_plot = pd.read_csv(eigenvec, sep='\t')
    pca_plot.drop(pca_plot.columns[[0]], axis=1, inplace=True)
    
    #Add PCA results to covariates file, then save
    
    covariates.IID = pd.to_numeric(covariates.IID)
    covariates = covariates.merge(pca_plot, left_on='IID', right_on='IID')
    folder = '/mnt/grid/ukbiobank/data/Application58510/skleeman/gwas_cystatinc/' + pop + '/covariates.tsv'
    covariates.to_csv(folder, sep="\t", index=False)